<a href="https://colab.research.google.com/github/PaulToronto/Stanford-Andrew-Ng-Machine-Learning-Specialization/blob/main/Overflow_safe_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overflow safe algorithm

## Imports

In [1]:
import numpy as np
import sympy as sym

## Functions

In [2]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Parameters
    ----------
    z : array_like
        A scalar or numpy array of any size.

    Returns
    -------
     g : array_like
         sigmoid(z)
    """
    z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0/(1.0+np.exp(-z))

    return g

In [3]:
def sigmoid_sym(z, clip=False):
    """
    Compute the sigmoid of z

    Parameters
    ----------
    z : array_like
        A scalar or numpy array of any size.

    Returns
    -------
     g : array_like
         sigmoid(z)
    """
    if clip:
        z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0 / (1.0 + sym.exp(-z))

    return g

In [4]:
def log_1pexp(x, maximum=20):
    ''' approximate log(1+exp^x)
        https://stats.stackexchange.com/questions/475589/numerical-computation-of-cross-entropy-in-practice
    Args:
    x   : (ndarray Shape (n,1) or (n,)  input
    out : (ndarray Shape matches x      output ~= np.log(1+exp(x))
    '''

    out  = np.zeros_like(x,dtype=float)
    i    = x <= maximum
    ni   = np.logical_not(i)

    out[i]  = np.log(1 + np.exp(x[i]))
    out[ni] = x[ni]
    return out

In [5]:
def log_1pexp_sym(x, maximum=20):
    ''' approximate log(1+exp^x)
        https://stats.stackexchange.com/questions/475589/numerical-computation-of-cross-entropy-in-practice
    Args:
    x   : (ndarray Shape (n,1) or (n,)  input
    out : (ndarray Shape matches x      output ~= np.log(1+exp(x))
    '''

    x = np.array(x, dtype='float64') # because it is a sympy Matrix

    out  = np.zeros_like(x,dtype=float)
    i    = x <= maximum
    ni   = np.logical_not(i)

    out[i]  = np.log(1 + np.exp(x[i]))
    out[ni] = x[ni]
    out = sym.Matrix(out)
    return out

## Compare algorithms

### Data

In [6]:
X = np.random.rand(6, 2)
y = np.array([0, 0, 0, 1, 1, 1]).reshape(-1, 1)
X, y

(array([[0.33653749, 0.07470506],
        [0.61771772, 0.46684572],
        [0.61398487, 0.77887118],
        [0.86671224, 0.91201079],
        [0.35166004, 0.13244731],
        [0.82014093, 0.21847606]]),
 array([[0],
        [0],
        [0],
        [1],
        [1],
        [1]]))

In [7]:
X_sym = sym.Matrix(X)
y_sym = sym.Matrix(y)
display(X_sym)
display(y_sym)

Matrix([
[0.336537489337893, 0.0747050566389048],
[0.617717723272549,  0.466845721010295],
[0.613984865025102,  0.778871177187558],
[0.866712235656763,  0.912010792871987],
[0.351660040892619,  0.132447309503545],
[0.820140934292399,  0.218476058604594]])

Matrix([
[0],
[0],
[0],
[1],
[1],
[1]])

In [8]:
w = np.random.rand(2, 1)
b = np.random.rand(1)[0]
w, b

(array([[0.63988059],
        [0.94059587]]),
 0.09001311477024132)

In [9]:
w_sym = sym.Matrix(w)
b_sym = sym.Matrix([b])[0]

### Safe vs. Unsafe

In [10]:
def safe_cost(X, y, w, b):
    m = X.shape[0]
    z = X @ w + b
    cost = -(y * z) + log_1pexp(z)
    cost = np.sum(cost) / m
    return cost

In [11]:
def safe_cost_sym(X, y, w, b):
    X = np.array(X, dtype='float64')
    y = np.array(y, dtype='float64')
    w = np.array(w, dtype='float64')
    b = np.array(b, dtype='float64')
    cost = safe_cost(X, y, w, b)
    return cost

In [12]:
def unsafe_cost(X, y, w, b):
    m = X.shape[0]
    f = sigmoid(X @ w + b)
    cost = (1 / m) * (np.dot(-y.T, np.log(f)) - np.dot((1-y).T, np.log(1-f)))
    cost = cost[0,0]
    return cost

In [13]:
def unsafe_cost_sym(X, y, w, b):
    X = np.array(X, dtype='float64')
    y = np.array(y, dtype='float64')
    w = np.array(w, dtype='float64')
    b = np.array(b, dtype='float64')
    cost = unsafe_cost(X, y, w, b)
    return cost

In [14]:
safe_cost(X, y, w, b) - unsafe_cost(X, y, w, b)

-1.1102230246251565e-16

In [15]:
safe_cost_sym(X_sym, y_sym, w_sym, b_sym) - unsafe_cost_sym(X_sym, y_sym, w_sym, b_sym)

-1.1102230246251565e-16

In [16]:
collect_all = []
for i in range(100):
    X = np.random.rand(6, 2)
    y = np.random.randint(0, 2, 6).reshape(-1, 1)
    w = np.random.rand(2, 1)
    b = np.random.rand(1)[0]
    difference = safe_cost(X, y, w, b) - unsafe_cost(X, y, w, b)
    collect_all.append(difference)

In [17]:
collect_all

[-1.1102230246251565e-16,
 0.0,
 -1.1102230246251565e-16,
 1.1102230246251565e-16,
 -1.1102230246251565e-16,
 0.0,
 0.0,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 2.220446049250313e-16,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 2.220446049250313e-16,
 1.1102230246251565e-16,
 -1.1102230246251565e-16,
 0.0,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 2.220446049250313e-16,
 0.0,
 1.1102230246251565e-16,
 0.0,
 1.1102230246251565e-16,
 0.0,
 0.0,
 0.0,
 2.220446049250313e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 0.0,
 0.0,
 0.0,
 0.0,
 -2.220446049250313e-16,
 -1.1102230246251565e-16,
 0.0,
 0.0,
 0.0,
 2.220446049250313e-16,
 -1.1102230246251565e-16,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 -1.1102230246251565e-16,
 0.0,
 0.0,
 1.1102230246251565e-16,
 0.0,
 -1.11022

In [18]:
sum(collect_all)

3.3306690738754696e-15

In [19]:
collect_all = []
for i in range(100):
    X = sym.Matrix(np.random.rand(6, 2), dtype='float64')
    y = sym.Matrix(np.random.randint(0, 2, 6).reshape(-1, 1), dtype='float64')
    w = sym.Matrix(np.random.rand(2, 1), dtype='float64')
    b = np.random.rand(1)[0]
    difference = safe_cost_sym(X, y, w, b) - unsafe_cost_sym(X, y, w, b)
    collect_all.append(difference)

In [20]:
collect_all

[-1.1102230246251565e-16,
 1.1102230246251565e-16,
 0.0,
 0.0,
 0.0,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 3.3306690738754696e-16,
 5.551115123125783e-17,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 2.220446049250313e-16,
 2.220446049250313e-16,
 -1.1102230246251565e-16,
 2.220446049250313e-16,
 -1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 0.0,
 0.0,
 0.0,
 -1.1102230246251565e-16,
 -1.1102230246251565e-16,
 -1.1102230246251565e-16,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 -5.551115123125783e-17,
 0.0,
 2.220446049250313e-16,
 0.0,
 1.1102230246251565e-16,
 0.0,
 2.220446049250313e-16,
 0.0,
 1.1102230246251565e-16,
 -1.1102230246251565e-16,
 0.0,
 0.0,
 0.0,
 2.220446049250313e-16,
 0.0,
 -1.1102230246251565e-16,
 1.1102230246251565e-16,
 -1.1102230246251565e-16,
 2.220446049250313e-16,
 4.440892098500626e-16,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-

In [21]:
sum(collect_all)

7.993605777301127e-15